# yt_xarray 

An xarray "accessor" interface to yt (for gridded data)

* Main docs: https://yt-xarray.readthedocs.io/en/latest/
* 2024 NASA Software Workshop Jupyter Book: [presentation](https://chrishavlin.github.io/NASASoftwareWorkshop2024/intro.html) and [code](https://github.com/data-exp-lab/yt_xarray_NASA_SMD_2024) 

Extra requirements:

```
pip install yt_xarray scipy xarray netCDF4 cartopy
```

```
wget https://yt2025data.hub.yt/geo/wUS-SH-2010_percent.nc
```



In [ ]:
import yt_xarray

ds = yt_xarray.open_dataset("IRIS/wUS-SH-2010_percent.nc")
type(ds)

In [ ]:
ds

In [ ]:
ds.dvp.isel({'depth': 1}).plot()

In [ ]:
# ds.yt.

In [ ]:
import cartopy.feature as cfeature

c=(125, 39., -110.625)
slc = ds.yt.SlicePlot('depth', 'dvs', center=c)

slc.set_mpl_projection(('Mollweide', (), {'central_longitude':c[2]}))

slc.set_log(('stream', 'dvs'), False)
slc.set_zlim(('stream', 'dvs'), -4, 4)
slc.set_cmap(('stream', 'dvs'), 'PRGn')
slc.render()  # render before making axes modifications 

slc.plots['stream', 'dvs'].axes.add_feature(cfeature.STATES)
 
slc.show()

In [ ]:
ds.yt.PhasePlot

just 3d arrays, can do:

In [ ]:
ds_yt = ds.yt.load_grid()

Other datasets

```python
ds_yt = ds.yt.load_grid(sel_dict={'time': 0})
```

In [ ]:
ds_yt.coordinates.axis_order

In [ ]:
ds_yt.coordinates.name

In [ ]:
import yt 
p = yt.PhasePlot(ds_yt, 'dvs', 'dvp', ('index', 'cell_volume'), weight_field=None)
p.set_log('dvs', False)
p.set_log('dvp', False)

In [ ]:
import cartopy.feature as cfeature

ys = ds_yt.arr([90., 45., -110], 'code_length')
wid = ds_yt.arr([ds_yt.domain_width[0].d, 10., 20.], 'code_length')

slc = yt.ProjectionPlot(ds_yt,
                      'depth', 
                      'dvs', 
                      center=ys, 
                      width=(wid[2], wid[1]), 
                      data_source=ds_yt.region(ys, ys-wid/2, ys+wid/2), 
                      weight_field=('index', 'cell_volume'),
                    )

slc.set_mpl_projection(('Mollweide', (), {'central_longitude':ys[2].d}))

slc.set_log(('stream', 'dvs'), False)
slc.set_cmap(('stream', 'dvs'), 'PRGn')
slc.render()  # render before making axes modifications 

slc.plots['stream', 'dvs'].axes.add_feature(cfeature.STATES)
 
slc.show()

## Embedded transformations

In [ ]:
from yt_xarray import transformations as tf

gc = tf.GeocentricCartesian(radial_type='depth', r_o=6371., use_neg_lons=True)

ds_yt_cart = tf.build_interpolated_cartesian_ds(
    ds,
    gc,
    fields = 'dvs' ,   
    grid_resolution = (32, 32, 32), 
    refine_grid=True,    
    refine_max_iters=2000,
    refine_min_grid_size=4,
    refine_by=4,
    interp_method='interpolate',
    refinement_method='signature_filter', # use  Berger and Rigoutsos 1991 
)

In [ ]:
slc = yt.SlicePlot(ds_yt_cart, 'x', ('stream', 'dvs'))
slc.set_log(("stream", "dvs"), False)
slc.set_cmap(("stream", "dvs"), "magma_r")
slc.set_zlim(("stream", "dvs"), -6, 6)
slc.annotate_cell_edges(alpha=0.2)
slc.annotate_grids(edgecolors=(1.,0,0,0))
slc.show()

In [ ]:
import numpy as np

def _dvs_not_nan(field, data):
    return ~np.isnan(data['dvs'])

ds_yt_cart.add_field(
    name=("stream", "dvs_not_nan"),
    function=_dvs_not_nan,
    sampling_type="local",
    units='',
    force_override=True,
)

In [ ]:
slc = yt.ProjectionPlot(ds_yt_cart, 'x', ('stream', 'dvs_not_nan'), weight_field=None)
slc.set_log(("stream", "dvs_not_nan"), False)
slc.set_cmap(("stream", "dvs_not_nan"), "octarine")
slc.annotate_grids(edgecolors=(1.,0,0,0))
slc.show()

Can volume render!

First: add a field of just the slow velocity anomalies (<0)

In [ ]:
import numpy
import unyt 

# add a nice field... volume rendering with observational data is hard...
# - fill in nans
# - full dvs is +/-, look at just -, make it + 
def _slow_vels(field, data):    
    dvs = data['dvs'].d.copy()
    dvs[np.isnan(dvs)] = 0.0
    dvs[dvs>0] = 0.0
    return unyt.unyt_array(np.abs(dvs),"")

ds_yt_cart.add_field(
    name=("stream", "slow_dvs"),
    function=_slow_vels,
    sampling_type="local",
    units="",    
)


In [ ]:
sc = yt.create_scene(ds_yt_cart, field=('stream', 'slow_dvs'))
cam = sc.add_camera(ds_yt_cart)

# transfer function 
source = sc[0]
source.tfh.set_bounds((0.1, 8))
source.tfh.set_log(True)

# adjust camera
cam.zoom(2)
cam.yaw(100*np.pi/180)
cam.roll(220*np.pi/180)
cam.rotate(30*np.pi/180)
cam.set_resolution((1000,1000))

sc.show(sigma_clip=5.)